In [1]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 5.1 MB/s 
     |████████████████████████████████| 6.6 MB 72.1 MB/s 
     |████████████████████████████████| 596 kB 76.1 MB/s 
     |████████████████████████████████| 84 kB 3.4 MB/s 
     |████████████████████████████████| 880 kB 77.6 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=4651efe40606266d6ba96461cd0e884aac6561c1bd22b42b521dd6be52e02be7
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm, tqdm_notebook
from transformers.optimization import AdamW, get_cosine_schedule_with_warmup
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel
import re

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
import os

n_devices = torch.cuda.device_count()
print(n_devices)

for i in range(n_devices):
    print(torch.cuda.get_device_name(i))

1
A100-SXM4-40GB


In [6]:
Q_TKN = "<usr>"
A_TKN = "<sys>"
BOS = '</s>'
EOS = '</s>'
MASK = '<unused0>'
SENT = '<unused1>'
PAD = '<pad>'

In [7]:
koGPT2_TOKENIZER = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
            bos_token=BOS, eos_token=EOS, unk_token='<unk>',
            pad_token=PAD, mask_token=MASK) 
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2').to(device)

Downloading:   0%|          | 0.00/2.69M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.98k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


Downloading:   0%|          | 0.00/490M [00:00<?, ?B/s]

In [8]:
PATH ="/content/drive/MyDrive/Model3(Text Generation)/model3_61.pt"

In [9]:
model.load_state_dict(torch.load(PATH))
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

In [12]:
test_data = pd.read_csv('/content/drive/MyDrive/asset/test_data/감성대화말뭉치AI데이터_평가용Wave_대본1만문장.csv')
test2_data = pd.read_csv('/content/drive/MyDrive/asset/test_data/emotion_data.csv')
test2_data = test2_data[['감정_대분류']]
test2_data

,감정_대분류
0,5
1,5
2,5
3,5
4,5
...,...
396,1
397,3
398,3
399,5


In [13]:
test_data = test_data[['사람문장1']]
test_data

,사람문장1
0,오늘 반장 선거에서 내가 반장이 됐어! 친구들이 날 믿어줘서 너무 고맙다.
1,오늘 시험에 친구가 알려준 문제가 나왔어. 결과가 좋아서 기쁘다.
2,오늘 대학 합격 통지를 받았어. 엄마가 너무 기뻐하는데 나는 눈물이 났어.
3,오늘 졸업식인데 선생님께서 아프셔서 오지 못하셨어. 그동안 감사하다는 말도 못했는데.
4,나 올림피아드에서 금상 받았어! 그동안 고생이 헛되지 않아서 너무 기뻐.
...,...
9995,소득이 너무 적어서 제대로 된 노후 대책을 할 수가 없어 힘들어.
9996,요즘 두통이 잦았는데 결국 이런 일이 생겼네. 난 건강한 줄 알았는데 충격받았어.
9997,친구가 돈이 행복의 전부가 아니라고 하더니 노후 준비를 전혀 안 한 거야. 어휴 정...
9998,아들이 갑자기 찾아왔는데 손자가 희귀병이래. 너무 두려워.


In [14]:
test1_data = []
for i in range(len(test_data)) :
  if i % 50 == 0 :
    test1_data.append(test_data.iloc[i][0])
test1_data

['오늘 반장 선거에서 내가 반장이 됐어! 친구들이 날 믿어줘서 너무 고맙다.',
 '할머니 집 마당에서 다 같이 고기를 구워먹었는데 참 행복했어.',
 '문제를 다 풀면 개념은 확실해질 거라 믿어.',
 '내 느낌에 이번 모의고사 평균 이 등급 받을 수 있을 거 같아!',
 '방금 내가 틀린 줄 알았던 문제의 답이 복수 정답 처리가 되었다는 말을 들었어. 너무 기뻐.',
 '고장 난 노트북을 고치는 대신 아빠가 최신형으로 바꿔 주셔서 기뻐.',
 '수능 성적이 안 좋아서 심란했는데 아빠가 위로해주셔서 기뻤어.',
 '친구들이 나를 괴롭히지 못하도록 선생님께서 감시하신다고 하셔서 얼마나 다행인지 몰라.',
 '오늘 기말고사를 봤는데 공부했던 부분에서 문제가 다 나와서 기분이 좋아.',
 '오늘 학교에 아빠가 작업복을 입고 면담을 오셨는데 옷차림이 신경이 쓰이더라.',
 '형이 없으면 나도 가족들에게 인정받을 수 있을 거야.',
 '별 것도 아닌 일로 내가 친구를 때렸어. 이제 생각하면 너무 부끄러워.',
 '우리 반에서 어느 한 명을 따돌림 시키는 일이 정말 옳은 일인가 싶어.',
 '내가 날씬해지면 인기가 많아질 것 같아.',
 '시험을 쳤는데 성적이 나빠 부모님께 성적표를 보여드리기 부끄러워.',
 '내가 너무 솔직하게 말을 한 걸까? 요즘 학교에 가기 싫어.',
 '정답지에 공부했던 내용을 전부 완벽히 쓰지 못해 나한테 화가 나.',
 '내 인사도 무시하던 친구가 내가 전교 일등 했다는 소식을 듣고 갑자기 친한 척을 해서 불쾌해.',
 '선생님은 친한 친구랑 나를 경쟁하게 만들어서 마음이 아파',
 '나한테 욕을 퍼붓는 애가 있는데 왜 성격이 나쁜데도 걔 옆에는 친구들이 많은지 궁금해.',
 '오늘 순간 욱해서 친구를 때렸어. 나 같은 구제 불능은 어디서도 안 받아줄 거야.',
 '오빠가 자기 인생 살겠다고 집을 나갔으면서 돈이 필요하니까 다시 들어왔어. 정말 구역질 나.',
 '친한 친구와 경쟁심이 생겨 공부 얘기를 잘하지 않게 되었어.',
 '노는 애

In [15]:
col_n = "사람문장1"
test1_datap = pd.DataFrame(test1_data, columns = [col_n])
test1_datap['사람문장1'] = test1_datap['사람문장1'].str.replace(pat=r'[^\w]', repl=r' ', regex=True)

test1_datap

,사람문장1
0,오늘 반장 선거에서 내가 반장이 됐어 친구들이 날 믿어줘서 너무 고맙다
1,할머니 집 마당에서 다 같이 고기를 구워먹었는데 참 행복했어
2,문제를 다 풀면 개념은 확실해질 거라 믿어
3,내 느낌에 이번 모의고사 평균 이 등급 받을 수 있을 거 같아
4,방금 내가 틀린 줄 알았던 문제의 답이 복수 정답 처리가 되었다는 말을 들었어 너...
...,...
195,지금은 건강하지만 나중에 아프게 되면 걱정인데 죽음이후에 사후세계가 있을까
196,자녀들이 암에 걸린 나를 서로 돌보지 않겠다고 싸우고 있어 환멸이 나
197,담배를 좀 더 일찍 끊었으면 좋았을 텐데 후회가 되네
198,은퇴 후에 해보고 싶었던 일로 첫 도전을 했던 마라톤에 참여하여 완주해서 정말 기뻐


In [16]:
test1_data = pd.concat([test1_datap, test2_data], axis=1)
test1_data

,사람문장1,감정_대분류
0,오늘 반장 선거에서 내가 반장이 됐어 친구들이 날 믿어줘서 너무 고맙다,5
1,할머니 집 마당에서 다 같이 고기를 구워먹었는데 참 행복했어,5
2,문제를 다 풀면 개념은 확실해질 거라 믿어,5
3,내 느낌에 이번 모의고사 평균 이 등급 받을 수 있을 거 같아,5
4,방금 내가 틀린 줄 알았던 문제의 답이 복수 정답 처리가 되었다는 말을 들었어 너...,5
...,...,...
396,NaN,1
397,NaN,3
398,NaN,3
399,NaN,5


In [18]:
t = 0
b_list = []
with torch.no_grad():
    while t != 200:
        q = test1_data.iloc[t][0]
        sent = str(7)
        a = ""
        while 1:
            input_ids = torch.LongTensor(koGPT2_TOKENIZER.encode(Q_TKN + q + SENT + sent + A_TKN + a)).unsqueeze(dim=0).to(device)
            pred = model(input_ids)
            pred = pred.logits
            gen = koGPT2_TOKENIZER.convert_ids_to_tokens(torch.argmax(pred, dim=-1).cpu().squeeze().numpy().tolist())[-1]
            if gen == EOS:
                break
            a += gen.replace("▁", " ")
        print(q)
        print("Chatbot > {}".format(a.strip()))
        b_list.append([q, a.strip(), sent])
        t = t + 1
b_list

오늘 반장 선거에서 내가 반장이 됐어  친구들이 날 믿어줘서 너무 고맙다 
Chatbot > 반장선거에서 친구들이 믿어줘서 고마우시군요
할머니 집 마당에서 다 같이 고기를 구워먹었는데 참 행복했어 
Chatbot > 다 같이 고기를 드셔서 행복하셨군요
문제를 다 풀면 개념은 확실해질 거라 믿어 
Chatbot > 사용자님이 생각한 대로 문제가 잘 풀렸으면 좋겠어요
내 느낌에 이번 모의고사 평균 이 등급 받을 수 있을 거 같아 
Chatbot > 이번 모의고사 성적을 바탕으로 다음 모의고사에서는 좋은 성적을 받길 바라요
방금 내가 틀린 줄 알았던 문제의 답이 복수 정답 처리가 되었다는 말을 들었어  너무 기뻐 
Chatbot > 틀린 줄 알았던 문제가 복수 정답이라니 기쁘시겠어요
고장 난 노트북을 고치는 대신 아빠가 최신형으로 바꿔 주셔서 기뻐 
Chatbot > 최신형 노트북을 고치는 대신 아빠가 최신형 노트북을 고치는 대신 아빠가 최신형 노트북을 고치는군요
수능 성적이 안 좋아서 심란했는데 아빠가 위로해주셔서 기뻤어 
Chatbot > 아빠가 위로해주셨다니 정말 기쁘시겠어요
친구들이 나를 괴롭히지 못하도록 선생님께서 감시하신다고 하셔서 얼마나 다행인지 몰라 
Chatbot > 선생님께서 감시하셔서 많이 힘드셨겠어요
오늘 기말고사를 봤는데 공부했던 부분에서 문제가 다 나와서 기분이 좋아 
Chatbot > 시험점수가 잘 나와서 기분이 좋으시군요
오늘 학교에 아빠가 작업복을 입고 면담을 오셨는데 옷차림이 신경이 쓰이더라 
Chatbot > 아빠가 옷차림이 신경이 쓰이셨군요
형이 없으면 나도 가족들에게 인정받을 수 있을 거야 
Chatbot > 가족들에게 인정받을 수 있기를 바라요
별 것도 아닌 일로 내가 친구를 때렸어  이제 생각하면 너무 부끄러워 
Chatbot > 친구에게 폭력을 당해 부끄러우시군요
우리 반에서 어느 한 명을 따돌림 시키는 일이 정말 옳은 일인가 싶어 
Chatbot > 어떤 일로 그렇게 생각하게 되었나요
내가 날씬해지면 인기가 많아질 것 같

[['오늘 반장 선거에서 내가 반장이 됐어  친구들이 날 믿어줘서 너무 고맙다 ', '반장선거에서 친구들이 믿어줘서 고마우시군요', '7'],
 ['할머니 집 마당에서 다 같이 고기를 구워먹었는데 참 행복했어 ', '다 같이 고기를 드셔서 행복하셨군요', '7'],
 ['문제를 다 풀면 개념은 확실해질 거라 믿어 ', '사용자님이 생각한 대로 문제가 잘 풀렸으면 좋겠어요', '7'],
 ['내 느낌에 이번 모의고사 평균 이 등급 받을 수 있을 거 같아 ',
  '이번 모의고사 성적을 바탕으로 다음 모의고사에서는 좋은 성적을 받길 바라요',
  '7'],
 ['방금 내가 틀린 줄 알았던 문제의 답이 복수 정답 처리가 되었다는 말을 들었어  너무 기뻐 ',
  '틀린 줄 알았던 문제가 복수 정답이라니 기쁘시겠어요',
  '7'],
 ['고장 난 노트북을 고치는 대신 아빠가 최신형으로 바꿔 주셔서 기뻐 ',
  '최신형 노트북을 고치는 대신 아빠가 최신형 노트북을 고치는 대신 아빠가 최신형 노트북을 고치는군요',
  '7'],
 ['수능 성적이 안 좋아서 심란했는데 아빠가 위로해주셔서 기뻤어 ', '아빠가 위로해주셨다니 정말 기쁘시겠어요', '7'],
 ['친구들이 나를 괴롭히지 못하도록 선생님께서 감시하신다고 하셔서 얼마나 다행인지 몰라 ',
  '선생님께서 감시하셔서 많이 힘드셨겠어요',
  '7'],
 ['오늘 기말고사를 봤는데 공부했던 부분에서 문제가 다 나와서 기분이 좋아 ', '시험점수가 잘 나와서 기분이 좋으시군요', '7'],
 ['오늘 학교에 아빠가 작업복을 입고 면담을 오셨는데 옷차림이 신경이 쓰이더라 ', '아빠가 옷차림이 신경이 쓰이셨군요', '7'],
 ['형이 없으면 나도 가족들에게 인정받을 수 있을 거야 ', '가족들에게 인정받을 수 있기를 바라요', '7'],
 ['별 것도 아닌 일로 내가 친구를 때렸어  이제 생각하면 너무 부끄러워 ', '친구에게 폭력을 당해 부끄러우시군요', '7'],
 ['우리 반에서 어느 한 명을 따돌림 시키는 

In [1]:
col_n1 = '질문'
col_n2 = '대답'
col_n3 = '감정'
finaltest_data = pd.DataFrame(b_list, columns = [col_n1, col_n2, col_n3])
finaltest_data

NameError: ignored

In [20]:
finaltest_data.to_csv("/content/drive/MyDrive/asset/test_data/final_data(no_emo).csv", mode='w')